### Get data from Cosmos db

In [1]:
# azure-core==1.26.2
# pip install azure-core
# azure-cosmos==4.3.0
# pip install azure-cosmos

SyntaxError: invalid syntax (1882036817.py, line 1)

In [2]:
from azure.cosmos import CosmosClient, ContainerProxy, DatabaseProxy, PartitionKey

In [3]:
endpoint = "https://bouman-earthquake-db.documents.azure.com:443/"
key = "kqshZbgJfMqCfLMaojjtNkXoYzKhe2BaaPgcrlDIdURoW1dWVPJkH9nsTXO2c1eaKXmQg20MwMSbACDb8XLizQ=="

In [4]:
def connect(endpoint:str, key:str) -> ContainerProxy:
    client = CosmosClient(endpoint, key)
    database_name = "raw_tweets"
    container_name = "tweets_db"
    partition_key = PartitionKey(path="/id")
    database:DatabaseProxy = client.create_database_if_not_exists(id=database_name)
    container:ContainerProxy = database.create_container_if_not_exists(id=container_name, partition_key=partition_key)
    return container

def query_cosmosdb(query:str):
    container = connect(endpoint=endpoint, key=key)
    items = list(container.query_items(
        query=query,
        enable_cross_partition_query=True
    ))
    return items 

In [5]:
#Conection to cosmos DB
raw_tweets = query_cosmosdb("SELECT * FROM r")  #Select all

### Translate tweet

In [ ]:
# deep-translator==1.9.2
# pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
#Translate dunction
def translate_tweet_f(dic:dict)-> dict :
    '''function that take dic and add a new colums who translate the text to english'''
    try:
        x = GoogleTranslator(source='auto', target='en').translate(dic['tweet_text'])
        dic['translate_text'] = x        
    except KeyError as err:
        dic['translate_text'] = ''
    return dic

### Apply formulas

In [ ]:

import json

In [ ]:
for raw_tweet in raw_tweets[0:5]:
    #transform in english 
    raw_tweet = translate_tweet_f(raw_tweet)
    eng_text = raw_tweet['translate_text']
    label = olivier_model(eng_text)
    #update dictionary
    raw_tweet["ml_label"]= label

In [ ]:
raw_tweets

In [ ]:
json_object = json.dumps(raw_tweets, indent=4)

with open("sample.json", "w") as outfile:
    outfile.write(json_object)